In [ ]:
import tensorflow as tf

print(tf.__version__)
print(tf.keras.__version__)

In [ ]:
import os
if os.path.exists('config/localconfig.py'):
    from config import localconfig as config
else:
    from config import config

In [ ]:
target_paths = os.listdir(config.BB_OBJECTS)

import random
random.shuffle(target_paths)

images, targets = [], []

In [ ]:
size = 1000
import json

for i in range(size):
    with open(os.path.join(config.BB_OBJECTS, target_paths[i])) as f:
        cones = json.load(f)

    image = tf.keras.utils.load_img(os.path.join(config.BB_IMAGES, target_paths[i][:-5]))
    for cone in cones:
        I_cone = tf.image.crop_to_bounding_box(image,
                                               cone['top_left']['y'],
                                               cone['top_left']['x'],
                                               cone['bottom_right']['y'] - cone['top_left']['y'],
                                               cone['bottom_right']['x'] - cone['top_left']['x'])
        images.append(tf.keras.utils.img_to_array(I_cone))
        targets.append(cone['class'])

In [ ]:
for i in range(len(images)):
    image = tf.keras.utils.array_to_img(images[i])
    image = image.resize((224, 224))
    images[i] = tf.keras.utils.img_to_array(image)

In [ ]:
import pandas as pd

targets = pd.get_dummies(targets)

In [ ]:
classes = targets.columns
targets = targets.values

In [ ]:
def get_class(dummy, classes):
    df = pd.DataFrame(columns=classes)
    df.loc[0] = dummy
    df = pd.from_dummies(df)
    return df.values[0, 0]

In [ ]:
import numpy as np

(x_train), (y_train) = (
    np.asarray(images[: int(len(images) * 0.8)]),
    np.asarray(targets[: int(len(targets) * 0.8)]),
)
(x_test), (y_test) = (
    np.asarray(images[int(len(images) * 0.8) :]),
    np.asarray(targets[int(len(targets) * 0.8) :]),
)

del images, targets

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(4, 4))
plt.imshow(x_train[0].astype("uint8"))
plt.title(get_class(y_train[0], classes))
plt.axis("off")

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(5, activation='relu'))

model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=[tf.keras.metrics.Accuracy()])

model.summary()

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                      patience=10)

history = model.fit(x_train[:200],
                    y_train[:200],
                    epochs=100,
                    validation_split=0.1,
                    callbacks=[es])

: 

In [ ]:
import time

model.save(f"conv2d_{int(time.time())}.h5", save_format="h5")